In [17]:
from phasic_tonic.DatasetLoader import DatasetLoader
from phasic_tonic.helper import get_metadata
from phasic_tonic.utils import get_start_end

import numpy as np
import pandas as pd
import pynapple as nap

from tqdm.auto import tqdm
from scipy.io import loadmat

fs_cbd = 2500
fs_os = 2500
fs_rgs = 1000

targetFs = 500
n_down_cbd = fs_cbd/targetFs
n_down_rgs = fs_rgs/targetFs
n_down_os = fs_os/targetFs

CONFIG_DIR = "/home/nero/phasic_tonic/data/dataset_loading.yaml"
OUTPUT_DIR1 = "/home/nero/phasic_tonic/data/analysis_output/whole_posttrial5/"
OUTPUT_DIR2 = "/home/nero/phasic_tonic/data/analysis_output/segmented_posttrial5/"

Datasets = DatasetLoader(CONFIG_DIR)
mapped_datasets = Datasets.load_datasets()

per_trial_stats = {}
per_epochs_stats = {}

def per_trial_analysis(metadata, state_epochs):
    for condition in metadata.keys():
        per_trial_stats[condition].append(metadata[condition])

    total_sleep = state_epochs["nrem"].tot_length() + state_epochs["intermediate"].tot_length() + state_epochs["rem"].tot_length()
    
    per_trial_stats['total_sleep_duration'].append(total_sleep)
    per_trial_stats['total_rem_duration'].append(state_epochs["rem"].tot_length())
    per_trial_stats['percentage_from_sleep'].append(state_epochs["rem"].tot_length()/total_sleep)
    per_trial_stats['counts_rem'].append(len(state_epochs['rem']))

def per_epochs_analysis(metadata, state_epochs):    
    for i, dur in enumerate(np.diff(state_epochs['rem'], 1)):
        for condition in metadata.keys():
            per_epochs_stats[condition].append(metadata[condition])
        per_epochs_stats['duration'].append(dur.item())
        per_epochs_stats['epoch_id'].append(i)

def get_state_epochs(hypno):
    state_epochs = {}
    sleep_states = {1:"wake", 3:"nrem",4:"intermediate",5:"rem"}

    for i in [1, 3, 4, 5]:
        start, end = get_start_end(sleep_states=hypno, sleep_state_id=i)
        state_epochs[sleep_states[i]] = nap.IntervalSet(start=start, end=end)
    return state_epochs


In [18]:
cbd_cnt = 0
rgs_cnt = 0
os_cnt = 0

# Count recordings belonging to CBD dataset
for name in mapped_datasets:
    metadata = get_metadata(name)
    if metadata['treatment'] == 0 or metadata['treatment'] == 1:
        cbd_cnt += 1
    elif metadata['treatment'] == 2 or metadata['treatment'] == 3:
        rgs_cnt += 1
    elif metadata['treatment'] == 4:
        os_cnt += 1

assert cbd_cnt == 170
assert rgs_cnt == 159
assert os_cnt == 210

# Whole post-trial 5

In [19]:
per_trial_stats = {
    'rat_id': [],
    'study_day': [],
    'condition': [],
    'treatment': [],
    'trial_num': [],
    'total_rem_duration':[],
    'total_sleep_duration':[],
    'percentage_from_sleep':[],
    'counts_rem':[]
    }

per_epochs_stats = {
    'rat_id': [],
    'study_day': [],
    'condition': [],
    'treatment': [],
    'trial_num': [],
    'epoch_id' : [],
    'duration' : []
}

with tqdm(mapped_datasets) as mapped_tqdm:
    for name in mapped_tqdm:
        metadata = get_metadata(name)
        mapped_tqdm.set_postfix_str(name)
        states_fname, _, _ = mapped_datasets[name]
        
        # Load the states
        hypno = loadmat(states_fname)['states'].flatten()

        # Convert each sleep states into dictionary of pynapple IntervalSets
        state_epochs = get_state_epochs(hypno)
        
        per_trial_analysis(metadata, state_epochs)
        per_epochs_analysis(metadata, state_epochs)

df_trials = pd.DataFrame(per_trial_stats)
df_epochs = pd.DataFrame(per_epochs_stats)

  0%|          | 0/539 [00:00<?, ?it/s]

/home/nero/anaconda3/envs/phasic_tonic/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/tmp/ipykernel_134376/2040524070.py:55: UserWarning: Some epochs have no duration
  state_epochs[sleep_states[i]] = nap.IntervalSet(start=start, end=end)
/tmp/ipykernel_134376/2040524070.py:39: RuntimeWarning: invalid value encountered in scalar divide
  per_trial_stats['percentage_from_sleep'].append(state_epochs["rem"].tot_length()/total_sleep)


In [20]:
df_trials

,rat_id,study_day,condition,treatment,trial_num,total_rem_duration,total_sleep_duration,percentage_from_sleep,counts_rem
0,5,8,HC,0,3,201.0,2552.0,0.078762,2
1,5,8,HC,0,1,0.0,479.0,0.000000,0
2,5,8,HC,0,2,398.0,2549.0,0.156140,4
3,5,8,HC,0,4,316.0,2190.0,0.144292,5
4,5,8,HC,0,5,864.0,5915.0,0.146069,13
...,...,...,...,...,...,...,...,...,...
534,11,4,OR,4,1,47.0,1435.0,0.032753,1
535,11,4,OR,4,4,412.0,2347.0,0.175543,3
536,11,4,OR,4,2,231.0,2025.0,0.114074,3
537,11,4,OR,4,5,1629.0,6498.0,0.250693,19


In [21]:
df_epochs

,rat_id,study_day,condition,treatment,trial_num,epoch_id,duration
0,5,8,HC,0,3,0,154.0
1,5,8,HC,0,3,1,47.0
2,5,8,HC,0,2,0,85.0
3,5,8,HC,0,2,1,56.0
4,5,8,HC,0,2,2,84.0
...,...,...,...,...,...,...,...
2151,11,4,OR,4,3,0,10.0
2152,11,4,OR,4,3,1,164.0
2153,11,4,OR,4,3,2,84.0
2154,11,4,OR,4,3,3,137.0


In [22]:
df_trials.to_csv(OUTPUT_DIR1+"rem_sleep_per_trial.csv", index=False)
df_epochs.to_csv(OUTPUT_DIR1+"rem_sleep_per_epoch.csv", index=False)

# Segmented post-trial 5

In [23]:
per_trial_stats = {
    'rat_id': [],
    'study_day': [],
    'condition': [],
    'treatment': [],
    'trial_num': [],
    'total_rem_duration':[],
    'total_sleep_duration':[],
    'percentage_from_sleep':[],
    'counts_rem':[]
    }

per_epochs_stats = {
    'rat_id': [],
    'study_day': [],
    'condition': [],
    'treatment': [],
    'trial_num': [],
    'epoch_id' : [],
    'duration' : []
}

with tqdm(mapped_datasets) as mapped_tqdm:
    for name in mapped_tqdm:
        metadata = get_metadata(name)
        mapped_tqdm.set_postfix_str(name)
        states_fname, _, _ = mapped_datasets[name]
        
        # Load the states
        hypno = loadmat(states_fname)['states'].flatten()
        
        if metadata["trial_num"] == '5':
            for i in range(4):
                metadata["trial_num"] = '5.' + str(i+1)
                hypno_segment = hypno[i*2700:(i+1)*2700]
                state_epochs = get_state_epochs(hypno_segment)

                per_trial_analysis(metadata, state_epochs)
                per_epochs_analysis(metadata, state_epochs)
        else:
            state_epochs = get_state_epochs(hypno)

            per_trial_analysis(metadata, state_epochs)
            per_epochs_analysis(metadata, state_epochs)

df_trials = pd.DataFrame(per_trial_stats)
df_epochs = pd.DataFrame(per_epochs_stats)

  0%|          | 0/539 [00:00<?, ?it/s]

/home/nero/anaconda3/envs/phasic_tonic/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/tmp/ipykernel_134376/2040524070.py:55: UserWarning: Some epochs have no duration
  state_epochs[sleep_states[i]] = nap.IntervalSet(start=start, end=end)
/tmp/ipykernel_134376/2040524070.py:39: RuntimeWarning: invalid value encountered in scalar divide
  per_trial_stats['percentage_from_sleep'].append(state_epochs["rem"].tot_length()/total_sleep)


In [24]:
df_trials

,rat_id,study_day,condition,treatment,trial_num,total_rem_duration,total_sleep_duration,percentage_from_sleep,counts_rem
0,5,8,HC,0,3,201.0,2552.0,0.078762,2
1,5,8,HC,0,1,0.0,479.0,0.000000,0
2,5,8,HC,0,2,398.0,2549.0,0.156140,4
3,5,8,HC,0,4,316.0,2190.0,0.144292,5
4,5,8,HC,0,5.1,481.0,2330.0,0.206438,5
...,...,...,...,...,...,...,...,...,...
858,11,4,OR,4,5.1,593.0,2141.0,0.276973,5
859,11,4,OR,4,5.2,571.0,2450.0,0.233061,8
860,11,4,OR,4,5.3,342.0,1264.0,0.270570,5
861,11,4,OR,4,5.4,122.0,641.0,0.190328,2


In [25]:
df_epochs

,rat_id,study_day,condition,treatment,trial_num,epoch_id,duration
0,5,8,HC,0,3,0,154.0
1,5,8,HC,0,3,1,47.0
2,5,8,HC,0,2,0,85.0
3,5,8,HC,0,2,1,56.0
4,5,8,HC,0,2,2,84.0
...,...,...,...,...,...,...,...
2177,11,4,OR,4,3,0,10.0
2178,11,4,OR,4,3,1,164.0
2179,11,4,OR,4,3,2,84.0
2180,11,4,OR,4,3,3,137.0


In [26]:
df_trials.to_csv(OUTPUT_DIR2+"rem_sleep_per_trial.csv", index=False)
df_epochs.to_csv(OUTPUT_DIR2+"rem_sleep_per_epoch.csv", index=False)